## Univariate and Multivariate Modeling and best fit - BASEBALL

### Create univariate linear regression to predict player salary

In [ ]:
import pandas as pd
import numpy as np
# from scipy.stats import 
import matplotlib.pyplot as plt
from sklearn import linear_model

PLAYER_SALARY = './baseball_data/Salaries.csv'
BATTING_STUFF = './baseball_data/Batting.csv'
batting = pd.read_csv(BATTING_STUFF, sep=',')
salary = pd.read_csv(PLAYER_SALARY, sep=',')
salary_list = np.array(salary['salary'])
home_runs_list = np.array(batting['HR'])
year_list = np.array(salary['yearID'])


In [ ]:
%matplotlib inline
salary.head(1), batting.head(1)

total_salaries = salary.groupby(["playerID"])["salary"].sum()
total_hrs = batting.groupby(["playerID"])["HR"].sum()

all_stats = pd.concat((total_hrs, total_salaries), axis=1)
all_stats.dropna(inplace = True)


plt.figure(figsize=(12, 4))
plt.scatter(all_stats.HR, all_stats.salary, edgecolor="None",
            s=5, c='k', alpha=0.2)
plt.yscale("log")
plt.xlabel("Home Runs", fontsize=12); plt.ylabel("Salary", fontsize=12)
plt.minorticks_on()
plt.xlim(-50)

xvals = np.array(all_stats.HR)[:, np.newaxis]
yvals = np.array(all_stats.salary)
regressor = linear_model.LinearRegression()
regressor.fit(xvals, yvals)
plt.scatter(xvals, regressor.predict(xvals), s= .5, color = 'orange')

plt.show()
all_stats[all_stats.HR == 0].count()

In [ ]:
from sklearn import cross_validation as cv
output = cv.KFold(len(all_stats), 10)

slopes, intercepts = [], []

for train_index, test_index in output:
    x_train, x_test = xvals[train_index], xvals[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    regressor.fit(x_train, y_train)
    slopes.append(regressor.coef_)
    intercepts.append(regressor.intercept_)
    
slope = np.mean(slopes)
intercept = np.mean(intercepts)

# regressor.coef_ = slope
# regressor.incept_ = intercept

print("Our model is:\n\tSalary = {} x N_HRs + {}".format(slope, intercept))

### This for loop divides our data into 10 sub-sections and performs a linear regression on each.  It then returns the average of the slopes of the regressions as well as the average of the intercepts.  These averages will be the m and y-intercept of our model, which we have now generalized by sampling 10 sub-groups.